# Import libraries

In [1]:
pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install setuptools wheel

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 10.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp311-cp311-macosx_10_9_x86_64.whl size=1116806 sha256=c36ffc3735f8fa0ee4153386ccf3547a6ccb191d70c12f1d71df7b73c6101ed5
  Stored in directory: /Users/riyagupta/Library/Caches/pip/wheels/f4/2b/26/e2a5eae55d3b7688995e66abe7f40473aac6c95ddd8ee174a8
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
import gc
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets

In [6]:
# Load Movies.csv
movies = pd.read_csv("ml-20m/movies.csv")

# Load Ratings.csv
ratings = pd.read_csv("ml-20m/ratings.csv")

# Load Tags.csv
tags = pd.read_csv("ml-20m/tags.csv")

# Merge tags and movies

In [7]:
for j in tags.movieId.unique():
   movies.loc[movies.loc[movies.movieId==j].index,'tags']=' '.join([i if type(i)==str else str(i) for i in tags.loc[ ( tags.movieId == j ),'tag'].unique().tolist() ])

# Content filtering method

In [8]:
from sklearn.model_selection import train_test_split
use,dontuse=train_test_split(ratings,test_size=0.995)

In [9]:
gc.collect()
user_movies_data=pd.pivot_table(use,index='movieId',columns='userId',values='rating',fill_value=0)
movies['tags']=movies['tags'].fillna('None')

In [10]:
tfidf=TfidfVectorizer()
tfidf_matrix=tfidf.fit_transform(movies['tags'])
tfidf_df=pd.DataFrame(tfidf_matrix.toarray(),index=movies.index.tolist())

In [12]:
svd=TruncatedSVD(n_components=19)
latent_matrix=svd.fit_transform(tfidf_df)

In [13]:
n=19
latent_matrix_1_df=pd.DataFrame(latent_matrix[:,0:n],index=movies['title'].tolist())

# Create Collab filter

In [14]:
svd=TruncatedSVD(n_components=20)
latent_matrix_2=svd.fit_transform(user_movies_data)

In [15]:
latent_matrix_2_df=pd.DataFrame(latent_matrix_2,index=[movies.loc[(movies.movieId==i),'title'].values[0] for i in (use['movieId'].unique())])

In [16]:
latent_matrix_1_df=latent_matrix_1_df.drop_duplicates()
latent_matrix_2_df=latent_matrix_2_df.drop_duplicates()

In [18]:
a = latent_matrix_2_df.copy()
for i in latent_matrix_1_df.index:
    if i not in latent_matrix_2_df.index:
        new_row_df = pd.DataFrame(np.zeros((1, 20)), index=[i], columns=np.arange(0, 20))
        a = pd.concat([a, new_row_df])

b = latent_matrix_1_df.copy()
for i in a.index:
    if i not in latent_matrix_1_df.index:
        new_row_df = pd.DataFrame(np.zeros((1, 20)), index=[i], columns=np.arange(0, 20))
        b = pd.concat([b, new_row_df])
        
a = a[~a.index.duplicated(keep='first')]

b = b[~b.index.duplicated(keep='first')]

# Hybrid model

In [21]:
def recommend_similar_movies(title):
    b_filled = b.fillna(0)
    a_filled = a.fillna(0)
    
    if title in b_filled.index:
        a_1 = np.array(b_filled.loc[title]).reshape(1, -1)
        score_content = cosine_similarity(b_filled, a_1).reshape(-1)
    else:
        score_content = np.zeros(b_filled.shape[0])
    
    if title in a_filled.index:
        a_2 = np.array(a_filled.loc[title]).reshape(1, -1)
        score_collab = cosine_similarity(a_filled, a_2).reshape(-1)
    else:
        score_collab = np.zeros(a_filled.shape[0])
    
    hybrid_score = (score_content + score_collab) / 2

    dictDF = {'content': score_content, 'collab': score_collab, 'hybrid': hybrid_score}
    similar_movies = pd.DataFrame(dictDF, index=a_filled.index)

    similar_movies.sort_values('hybrid', ascending=False, inplace=True)
    return similar_movies.head()


In [22]:
recommend_similar_movies('Toy Story (1995)')

,content,collab,hybrid
"Reflecting Skin, The (1990)",0.886805,0.852208,0.869506
"20,000 Leagues Under the Sea (1954)",0.958696,0.776141,0.867419
Philadelphia (1993),0.893544,0.764527,0.829035
"Son's Room, The (Stanza del figlio, La) (2001)",0.918878,0.737515,0.828196
"Family Thing, A (1996)",0.927662,0.715633,0.821648


In [23]:
recommend_similar_movies('Mission: Impossible II (2000)')

,content,collab,hybrid
Cleopatra (1963),0.835018,0.998090,0.916554
White Squall (1996),0.762218,0.999046,0.880632
13 Assassins (Jûsan-nin no shikaku) (2010),0.724798,0.996510,0.860654
Last Orders (2001),0.756595,0.954578,0.855587
Samurai Rebellion (Jôi-uchi: Hairyô tsuma shimatsu) (1967),0.723456,0.984386,0.853921
